In [58]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Set Up Environment

In [131]:
connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://13.57.193.25:9000/",        "nid": 3}}

env = connections["custom"] # mainnet, yeouido, euljiro, pagoda, custom

In [147]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx6927160050f452ae32163c895c08e0171aa6073c'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx4c94c5802cd7c4c9b9e1a5eef96c71cbb024f8f7'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx112c953f6f505f688401af0daf0345cb87ef5492'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxf90da35301d655e7fc760d5576b449a534f1987f'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxcdc859b49084b4bfe63d8efb711f71bdc1482d35'}}

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [133]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [134]:
wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")

In [254]:
deploy = ['loans', 'staking', 'dex', 'sicx', 'icd']
# contracts = {}
for directory in {"core_contracts", "token_contracts"}:
    with os.scandir(directory) as it:
        for file in it:
            archive_name = directory + "/" + file.name
            if file.is_dir() and file.name in deploy:
                make_archive(archive_name, "zip", directory, file.name)
                contracts[file.name]['zip'] = archive_name + '.zip'
contracts

{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx5315f295b740bbc8e9dc6cf1a049b7b3477a2778'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx198def8637efd2606d4634e2a8fff0f7659eea33'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx3d8e345681f2507f194f2b22846a0f7acc5da020'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxd9512d1898d0c8cc1376aa93b3d107903745de29'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxe270fe4e3e1d98e7946c0269ea5fa78e8f8f502f'}}

In [273]:
ro_methods = ["name", "symbol", "decimals", "totalSupply", "getAdmin"]

In [269]:
ro_methods = ["name", "get_sICX_address", "get_rate"]

In [271]:
ro_methods = ["name", "get_oracle_address", "get_available_assets"]

In [274]:
results = {}
for method in ro_methods:
    call = CallBuilder().from_(wallet.get_address())\
                        .to(contracts['sicx']['SCORE'])\
                        .method(method)\
                        .build()
    result = icon_service.call(call)
    print(f'{method}: {result}')
    results[method] = result

name: StakedICX
symbol: sICX
decimals: 0x12
totalSupply: 0x0
getAdmin: cx4c94c5802cd7c4c9b9e1a5eef96c71cbb024f8f7


## Deploy All SCOREs

In [264]:
step_limit = 4000100000

for score, loc in contracts.items():
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


...

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx6927160050f452ae32163c895c08e0171aa6073c'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx4c94c5802cd7c4c9b9e1a5eef96c71cbb024f8f7'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx112c953f6f505f688401af0daf0345cb87ef5492'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxf90da35301d655e7fc760d5576b449a534f1987f'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxcdc859b49084b4bfe63d8efb711f71bdc1482d35'}}

## Update a SCORE

In [250]:
contract = 'loans'
update = contracts[contract]['SCORE']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contracts[contract]["zip"]))\
    .build()

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x637abc6378b0c4f38f94efc599e985d652439de7f31e33f13d9d61b75b72c3e8',
 'blockHeight': 36647,
 'blockHash': '0xf202d7471d48b1bf63fd7105f28a98a7c5cfb305dcf82efd04ae0ddea953bfa7',
 'txIndex': 0,
 'to': 'cx5315f295b740bbc8e9dc6cf1a049b7b3477a2778',
 'scoreAddress': 'cx5315f295b740bbc8e9dc6cf1a049b7b3477a2778',
 'stepUsed': 2002392800,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 2002392800,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Set parameters

In [157]:
contracts['sicx']['SCORE']

'cxd9512d1898d0c8cc1376aa93b3d107903745de29'

In [265]:
settings = [{'contract': 'sicx', 'method': 'setAdmin', 'params':{'_admin': contracts['staking']['SCORE']}},
            {'contract': 'staking', 'method': 'set_sICX_address', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'loans', 'method': 'set_sicx_address', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'loans', 'method': 'add_asset', 'params':{'_token_address': contracts['icd']['SCORE']}},
            {'contract': 'loans', 'method': 'set_staking_contract', 'params':{'_address': contracts['staking']['SCORE']}},
            {'contract': 'loans', 'method': 'toggle_loans_on', 'params':{}}]


In [266]:
for sett in settings:
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


...

{'txHash': '0xa2f7f11fe7d622877bba09fa8ea5f7504f3909c1ed9f33461bc0affa65266332', 'blockHeight': 33, 'blockHash': '0x60c61de1aaefa47cb4d550b3186d1d34ac00c868b2eeed6f01860cc75a60aea3', 'txIndex': 0, 'to': 'cx6927160050f452ae32163c895c08e0171aa6073c', 'stepUsed': 1066800, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1066800, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'txHash': '0x1fef13fda0d4016b1e600f40763fc439b1409dbd5b53cd6c01a484746f6387de', 'blockHeight': 34, 'blockHash': '0x4132f8d47dd3c20b06084dd8d6f05ffe3a76a065e94e097e292f7c9eb1f503c3', 'txIndex': 0, 'to': 'cx6927160050f452ae32163c895c08e0171aa6073c', 'stepUsed': 1031600, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1031600, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [276]:
params = {'_symbol': 'USD'}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method('get_price_in_icx')\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x906f8519226314e004d6d97d418f1af18b2fa60a3daf9ef523f0ffdc706c55b8', 'blockHeight': 94, 'blockHash': '0x4b6c859292ed48d76eca529495178752169708f4d660d7b19ad94e650f06f876', 'txIndex': 0, 'to': 'cx6927160050f452ae32163c895c08e0171aa6073c', 'stepUsed': 1033800, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1033800, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [143]:
wallet.get_address()

'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'

## Depost ICX and Mint ICD Loan

In [275]:
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_for\": \"hxe7af5fcfd8dfc67530a01a0e403882687528dfcb\", \"_asset\": \"ICD\", \"_amount\": 400000000000000000000}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(10000 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0xfe5dccb30328e1c1fe150aaa579377a64111ae8e0572ac910695dfa5e0770fe8', 'blockHeight': 76, 'blockHash': '0xfcfd74e2a59a3bb9e80822a0d576f4cfa2acdf8049bd8d1e8dde8c8ab4884284', 'txIndex': 0, 'to': 'cx4c94c5802cd7c4c9b9e1a5eef96c71cbb024f8f7', 'stepUsed': 1137750, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1137750, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0